In [1]:
import openai
import langchain
import pinecone
import os

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAI


d:\Dev\PineconeDB1\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
def read_doc(directory):
    all_docs = []
    
    for filename in os.listdir(directory):
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            loader = PyPDFLoader(file_path)
            docs = loader.load()
            all_docs.extend(docs)

    return all_docs

In [4]:
doc = read_doc("documents/")
doc

[Document(metadata={'producer': 'Skia/PDF m144 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'ONE STOP CENTRE', 'source': 'documents/onestopcenter.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='What  is  a  One  Stop  Centre?  One  Stop  Centres  (OSCs)  are  intended  to  support  women  affected  by  violence,  in  private  \nand\n \npublic\n \nspaces,\n \nwithin\n \nthe\n \nfamily,\n \ncommunity\n \nand\n \nat\n \nthe\n \nworkplace.\n \nWomen\n \nfacing\n \nphysical,\n \nsexual,\n \nemotional,\n \npsychological\n \nand\n \neconomic\n \nabuse,\n \nirrespective\n \nof\n \nage,\n \nclass,\n \ncaste,\n \neducation\n \nstatus,\n \nmarital\n \nstatus,\n \nrace\n \nand\n \nculture\n \nwill\n \nbe\n \nfacilitated\n \nwith\n \nsupport\n \nand\n \nredressal.\n \nIt\n \nis\n \nintegrated\n \nwith\n \nwomen\n \nHelp\n \nline\n \nnumber\n \n181\n  What  are  the  services  delivered  in  the  OSC?   Emergency  Response  and  Rescue  Services  Medic

In [5]:
len(doc)

3

In [6]:
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
  text_splitter=RecursiveCharacterTextSplitter(
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap
  )
  texts=text_splitter.split_documents(docs)
  return texts

In [7]:
documents=chunk_data(docs=doc)
documents

[Document(metadata={'producer': 'Skia/PDF m144 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'ONE STOP CENTRE', 'source': 'documents/onestopcenter.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='What  is  a  One  Stop  Centre?  One  Stop  Centres  (OSCs)  are  intended  to  support  women  affected  by  violence,  in  private  \nand\n \npublic\n \nspaces,\n \nwithin\n \nthe\n \nfamily,\n \ncommunity\n \nand\n \nat\n \nthe\n \nworkplace.\n \nWomen\n \nfacing\n \nphysical,\n \nsexual,\n \nemotional,\n \npsychological\n \nand\n \neconomic\n \nabuse,\n \nirrespective\n \nof\n \nage,\n \nclass,\n \ncaste,\n \neducation\n \nstatus,\n \nmarital\n \nstatus,\n \nrace\n \nand\n \nculture\n \nwill\n \nbe\n \nfacilitated\n \nwith\n \nsupport\n \nand\n \nredressal.\n \nIt\n \nis\n \nintegrated\n \nwith\n \nwomen\n \nHelp\n \nline\n \nnumber\n \n181'),
 Document(metadata={'producer': 'Skia/PDF m144 Google Docs Renderer', 'creator': 'PyPDF', 'creationdat

In [8]:
from pydantic import SecretStr
embeddings=OpenAIEmbeddings(api_key=SecretStr(os.environ['OPENAI_API_KEY']))
embeddings 

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000002A0311C3A90>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000002A0319B0C10>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [9]:
vectors=embeddings.embed_query("What is Domestic Violence?")
len(vectors)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}